In [1]:
import pandas as pd

# Specify the file path
file_path = "Salaries RME (Nov-15 To Sep-23).xlsx"

# Read the Excel file into a Pandas DataFrame
df = pd.read_excel(file_path)

# Display the first few rows of the DataFrame to check the data
print(df.head())

       month                 project     amount
0 2015-11-01               Equipment   91446.06
1 2015-11-01  Procurement Department  161983.17
2 2015-11-01          Administration  105386.33
3 2015-11-01                  Stores   22338.38
4 2015-11-01         Human Resources   56122.06


In [2]:
# Group the data by the 'project' column and sum the 'amount' for each project
project_totals = df.groupby('project')['amount'].sum().reset_index()

# Specify the output Excel file path
output_file = "Project_Salary_Totals.xlsx"

# Save the project totals to a new Excel file
project_totals.to_excel(output_file, index=False)

print("Project salary totals saved to", output_file)

Project salary totals saved to Project_Salary_Totals.xlsx


In [3]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

In [ ]:
# Aggregate data for all projects per month
df_all_projects = df.groupby('month')['amount'].sum().reset_index()
df_all_projects['project'] = 'All Projects'

# Concatenate the data for all projects with the original data
df = pd.concat([df, df_all_projects], ignore_index=True)

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the web app
app.layout = html.Div([
    dcc.Dropdown(
        id='project-dropdown',
        options=[{'label': 'All Projects', 'value': 'All Projects'}] + [{'label': project, 'value': project} for project in df['project'].unique()],
        value='All Projects'
    ),
    html.Div([
        html.Div(id='total-salary-text', style={'fontSize': 24, 'fontWeight': 'bold', 'textAlign': 'center'}),
        html.Div(id='latest-month-amount-text', style={'fontSize': 18, 'fontWeight': 'bold', 'textAlign': 'center'}),
    ]),
    dcc.Graph(id='salary-line-plot')
])

# Define a function to calculate the total salary amount for a selected project
def calculate_total_salary(selected_project):
    if selected_project == "All Projects":
        filtered_df = df_all_projects
        title = "Salaries for All Projects"
    else:
        filtered_df = df[df['project'] == selected_project]
        title = f'Salaries for {selected_project}'
    total_salary = filtered_df['amount'].sum()
    return filtered_df, total_salary, title

# Define the callback to update the plot and total salary text based on the selected project
@app.callback(
    [Output('salary-line-plot', 'figure'),
     Output('total-salary-text', 'children'),
     Output('latest-month-amount-text', 'children')],
    Input('project-dropdown', 'value')
)
def update_graph(selected_project):
    filtered_df, total_salary, title = calculate_total_salary(selected_project)
    
    if selected_project == "All Projects":
        latest_month_amount = df_all_projects['amount'].iloc[-1]
        latest_month_amount_text = f"Amount for Latest Month: {latest_month_amount:,.2f} EGP"
    else:
        latest_month = filtered_df['month'].max()
        latest_amount = filtered_df[filtered_df['month'] == latest_month]['amount'].values[0]
        latest_month_amount_text = f"Amount for Latest Month: {latest_amount:,.2f} EGP"
    
    fig = px.line(filtered_df, x='month', y='amount', title=title)
    total_salary_text = f"Total Salary: {total_salary:,.2f} EGP"
    
    return fig, total_salary_text, latest_month_amount_text

if __name__ == '__main__':
    app.run_server(debug=True)